In [1]:
import pandas as pd
from path_planner import *
from lib.dds.dds import *

In [2]:
df = pd.read_csv('coordinates.csv', sep=';')

In [3]:
#planner = PathPlanner(coordinates = [(x, y) for x, y in zip(df['X'], df['Y'])], edges = [(0, 1), (1, 2)])

In [4]:
#planner = PathPlanner([[5.11, -11.89], [5.11, -0.781], [5.11, 10.12], [6.56, 13.44]], [[0, 1], [1, 2], [2, 3]])
robot = Multirotor(x_origin = 12.117, y_origin=6.371, z_origin=0.060)
#move_list = planner.create_path(robot, 0, 2)

In [5]:
move_list = []

move_list.insert(0, ZMovement(robot, 1.2))
move_list.append(XYMovement(robot, 11.056, 4.197 ))
move_list.append(ZMovement(robot, 0.9))
move_list.append(AttachMovement(1))
move_list.append(AttachMovement(0))
move_list.append(ZMovement(robot, 1.2))
move_list.append(XYMovement(robot, 12, 6.4 ))
move_list.append(ZMovement(robot, 0.9))
move_list.append(AttachMovement(2))
move_list.append(AttachMovement(0))
move_list.append(ZMovement(robot, 2))

In [ ]:
dpvz = DataPlotter()
dpvz.set_x("time (seconds)")
dpvz.add_y("target_vx", "target_vx")
dpvz.add_y("current_vx", "current_vx")

dpz = DataPlotter()
dpz.set_x("time (seconds)")
dpz.add_y("target_x", "target_x")
dpz.add_y("current_x", "current_x")

move_command = move_list.pop(0)
move_command.start()

dds = DDS()
dds.start()

dds.subscribe(['start','tick', 'X', 'Y', 'Z', 'TX', 'TY', 'TZ', 'VX', 'VY', 'VZ', 'WX', 'WY', 'WZ'])


t = Time(use_fake_time=False)
t.start()

while True:
    dds.wait('tick')
    delta_t = t.elapsed()
    
    z = dds.read('Z')
    vz = dds.read('VZ')
    
    x = dds.read('X')
    vx = dds.read('VX')
    
    y = dds.read('Y')
    vy = dds.read('VY')
    
    roll = dds.read('TX')
    roll_rate = dds.read('WX')
    
    pitch = dds.read('TY')
    pitch_rate = dds.read('WY')
    
    if type(move_command) == AttachMovement:
        dds.publish('attached', move_command.evaluate(delta_t), DDS.DDS_TYPE_INT)
        print("ATTACH MOVEMENT")
        
        if move_list == []:
                print("FINITO")
                break
            
        move_command = move_list.pop(0)
        move_command.start()

    else:  
        (f1, f2, f3, f4) = robot.evaluate(delta_t, z, vz, x, vx, y, vy, roll, roll_rate, pitch, pitch_rate)
        move_command.evaluate(delta_t)
        
        if move_command.movement_done():
            print("FINITO MOVIMENTO")
            
            if move_list == []:
                print("FINITO")
                break
                
            move_command = move_list.pop(0)
            move_command.start()

    dds.publish('f1', f1, DDS.DDS_TYPE_FLOAT)
    dds.publish('f2', f2, DDS.DDS_TYPE_FLOAT)
    dds.publish('f3', f3, DDS.DDS_TYPE_FLOAT)
    dds.publish('f4', f4, DDS.DDS_TYPE_FLOAT)
    
    dpvz.append_x(t.get())
    dpvz.append_y("target_vx", robot.vz_target)
    dpvz.append_y("current_vx", vz)

    dpz.append_x(t.get())
    dpz.append_y("target_x", robot.z_target)
    dpz.append_y("current_x", z)


dds.publish('f1', 0, DDS.DDS_TYPE_FLOAT)
dds.publish('f2', 0, DDS.DDS_TYPE_FLOAT)
dds.publish('f3', 0, DDS.DDS_TYPE_FLOAT)
dds.publish('f4', 0, DDS.DDS_TYPE_FLOAT)

dds.stop()
dpvz.plot()
dpz.plot()

print("Done")

FINITO MOVIMENTO


In [ ]:
move_command

In [ ]:
robot.x_target

12.0